<a href="https://colab.research.google.com/github/D34dP0oL/4216_Biomedical_DS_and_AI/blob/main/Sheet3/Assignment3_Solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import math

## Biomedical Data Science & AI

## Assignment 3

#### Group members:  Fabrice Beaumont, Fatemeh Salehi, Genivika Mann, Helia Salimi, Jonah

#### Exercise 1 - Probability
The amount of wine bottles sold in a shop follows a Poisson distribution with *180*
bottles per week (6 days). If $C$ is the **random variable for bottles per day**, how is:

#### 1.a. The probability that the shop will only sell 20 bottles per day?

The probability distribution of a Poisson random variable $X$ (here amount of sold wine bottles over time) is given by
$$ \mathbb{P}[X=x] = \frac{\exp(-\mu) \mu^x}{x!} $$


The given setting gives the following means: $\mu_{\text{week}} = 180 \ \implies \ \mu_{\text{day}} = 180/6 = 30$. 

Thus the probability to sell $20$ bottles per day is:

$\begin{aligned} \mathbb{P}[X=20] &= \frac{\exp(-\mu_{\text{day}}) \mu_{\text{day}}^{20}}{20!}\\
&= \frac{\exp(-\mu_{\text{day}}) \mu_{\text{day}}^{20}}{20!} \\
&= \frac{\exp(-30) 30^{20}}{20!} \qquad \approx \text{well ... tiny. This can't be right - right?}\\
\end{aligned}$


In [12]:
print(f"{np.exp(-30)* np.power(30, 20) / math.factorial(20)}")

-1.6755778325549218e-13


#### 1.b. The probability that the demand is more than average for a particular day?

**No statement about the demand was made!** We assume this is equal to the number of sold bottles. Since $\mu_{\text{day}}$ is the expected/average abount of sold bottles per day, the probability to sell more than this is:

$\begin{aligned} \mathbb{P}[X> \mu_{\text{day}}] = 1-\mathbb{P}[X\le \mu_{\text{day}}] &= 1-\sum_{i=0}^{\mu_{\text{day}}} \frac{\exp(-\mu_{\text{day}}) \mu_{\text{day}}^{i}}{i!}\\
&= \dots 
\end{aligned}$

#### 1.c. The expected number of units per day $\mathbb{E}[C]$?

#### 1.d. What is $\text{Var}[C]$?

#### 1.e. The standard deviation of $C$?

#### Exercise 2 - Hypothesis testing
This exercise illustrates a gene expression data set with its normally distributed values.
Consider the gene expression data of the Golub dataset. Load the file `golub.csv`. 
It contains gene expression data of 3051 genes from 38 tumor mRNA samples. The expression data is organized in a matrix where rows correspond to genes and columns to samples. The tumor class of the columns is given in the file `golub.cl`. The names of the genes (rows) are given in `golub.gnames`.

#### 2.1. Calculate the sample mean of all genes $\beta$ in the pooled expression matrix. Use these means to determine the overall mean $\beta_0$ by just taking the average.